In [1]:
import torch
import torchvision
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
import torchvision.models as models
import torchvision.transforms as transforms
import torch.optim as optim

In [2]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

dataset = CIFAR10(root='data/',  download=True, train=True, transform=transform)
test_dataset = CIFAR10(root='data/', download=True, train=False, transform=transform)

classes = dataset.classes

torch.manual_seed(1)
val_size = 5000
train_size = len(dataset) - val_size
train_data, val_data = random_split(dataset, [train_size, val_size])
batch_size=4

train_loader = DataLoader(train_data, batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_data, batch_size, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size, shuffle=False, num_workers=2)

In [3]:
alexnet = models.alexnet(pretrained=True)
alexnet.classifier[4] = nn.Linear(4096,1024)
alexnet.classifier[6] = nn.Linear(1024,10)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

alexnet.to(device)

cuda:0


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [4]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(alexnet.parameters(), lr=0.001)
for epoch in range(1): # 1 for now to see an initial results
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()
        output = alexnet(inputs)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 2000 == 1999:    
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training of AlexNet')

[1,  2000] loss: 1.057
[1,  4000] loss: 0.733
[1,  6000] loss: 0.643
[1,  8000] loss: 0.597
[1, 10000] loss: 0.568
Finished Training of AlexNet


In [5]:
correct = 0
total = 0

num_correct = 0
num_samples = 0
with torch.no_grad():
    for data in test_loader:
        images, targets = data[0].to(device), data[1].to(device)        
        scores = alexnet(images)
        _, predictions = torch.max(scores.data, 1)
        num_correct += (predictions == targets).sum().item()
        num_samples += predictions.size(0)
        
print(f"Got {num_correct} / {num_samples} with accuracy {float(num_correct) / float(num_samples) * 100:.2f}")

Got 8108 / 10000 with accuracy 81.08
